In [1]:

!pip install torchgeometry


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip show torch


Name: torch
Version: 2.0.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, catalyst, easyocr, fastai, kornia, pytorch-ignite, pytorch-lightning, stable-baselines3, timm, torchaudio, torchdata, torchgeometry, torchmetrics, torchtext, torchvision


In [3]:

# from torchgeometry.losses import one_hot
import os
import os.path as osp
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import time
import imageio
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Resize, PILToTensor, ToPILImage, Compose, InterpolationMode
from collections import OrderedDict
import wandb
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import glob
import multiprocessing.pool as mpp
import multiprocessing as mp
import argparse
import torch
import albumentations as albu
from torchvision.transforms import (Pad, ColorJitter, Resize, FiveCrop, RandomResizedCrop,
                                    RandomHorizontalFlip, RandomRotation, RandomVerticalFlip)
import random
import math
import numbers
from PIL import Image, ImageOps, ImageEnhance
import numpy as np
import random
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion
from scipy.ndimage import maximum_filter
from tqdm import tqdm
import pytorch_lightning as pl
# from pytorch_lightning.callbacks import Model
import os
import torch
from torch import nn
import cv2
import numpy as np
import argparse
from pathlib import Path

from pytorch_lightning.loggers import CSVLogger
import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Parameters

In [5]:

num_classes = 6

# Number of epoch
epochs = 25

# Hyperparameters for training 
learning_rate = 1e-03
batch_size = 16
display_step = 2

# Model path
checkpoint_path = '/kaggle/working/unetplusplus_tune.pth'
pretrained_path = "/kaggle/input/model2/unet_model.pth"
# Initialize lists to keep track of loss and accuracy
loss_epoch_array = []
train_accuracy = []
test_accuracy = []
valid_accuracy = []

In [6]:
SEED = 42


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(42)

ImSurf = np.array([255, 255, 255])  # label 0
Building = np.array([255, 0, 0]) # label 1
LowVeg = np.array([255, 255, 0]) # label 2
Tree = np.array([0, 255, 0]) # label 3
Car = np.array([0, 255, 255]) # label 4
Clutter = np.array([0, 0, 255]) # label 5
Boundary = np.array([0, 0, 0]) # label 6
num_classes = 6





def pv2rgb(mask):
    h, w = mask.shape[0], mask.shape[1]
    mask_rgb = np.zeros(shape=(h, w, 3), dtype=np.uint8)
    mask_convert = mask[np.newaxis, :, :]
    mask_rgb[np.all(mask_convert == 3, axis=0)] = [0, 255, 0]
    mask_rgb[np.all(mask_convert == 0, axis=0)] = [255, 255, 255]
    mask_rgb[np.all(mask_convert == 1, axis=0)] = [255, 0, 0]
    mask_rgb[np.all(mask_convert == 2, axis=0)] = [255, 255, 0]
    mask_rgb[np.all(mask_convert == 4, axis=0)] = [0, 204, 255]
    mask_rgb[np.all(mask_convert == 5, axis=0)] = [0, 0, 255]
    return mask_rgb
def label2rgb(mask):
    h, w = mask.shape[0], mask.shape[1]
    mask_rgb = np.zeros(shape=(h, w, 3), dtype=np.uint8)
    mask_convert = mask[np.newaxis, :, :]
    mask_rgb[np.all(mask_convert == 3, axis=0)] = [0, 255, 0]
    mask_rgb[np.all(mask_convert == 0, axis=0)] = [255, 255, 255]
    mask_rgb[np.all(mask_convert == 1, axis=0)] = [255, 0, 0]
    mask_rgb[np.all(mask_convert == 2, axis=0)] = [255, 255, 0]
    mask_rgb[np.all(mask_convert == 4, axis=0)] = [0, 204, 255]
    mask_rgb[np.all(mask_convert == 5, axis=0)] = [0, 0, 255]
    return mask_rgb



def car_color_replace(mask):
    mask = cv2.cvtColor(np.array(mask.copy()), cv2.COLOR_RGB2BGR)
    mask[np.all(mask == [0, 255, 255], axis=-1)] = [0, 204, 255]

    return mask


def rgb_to_2D_label(_label):
    _label = _label.transpose(2, 0, 1)
    label_seg = np.zeros(_label.shape[1:], dtype=np.uint8)
    label_seg[np.all(_label.transpose([1, 2, 0]) == ImSurf, axis=-1)] = 0
    label_seg[np.all(_label.transpose([1, 2, 0]) == Building, axis=-1)] = 1
    label_seg[np.all(_label.transpose([1, 2, 0]) == LowVeg, axis=-1)] = 2
    label_seg[np.all(_label.transpose([1, 2, 0]) == Tree, axis=-1)] = 3
    label_seg[np.all(_label.transpose([1, 2, 0]) == Car, axis=-1)] = 4
    label_seg[np.all(_label.transpose([1, 2, 0]) == Clutter, axis=-1)] = 5
    label_seg[np.all(_label.transpose([1, 2, 0]) == Boundary, axis=-1)] = 6
    return label_seg



    

 # Split Data

 # Dataloader

In [7]:
CLASSES = ('ImSurf', 'Building', 'LowVeg', 'Tree', 'Car', 'Clutter')
class_label={0:'ImSurf', 1:'Building',2: 'LowVeg', 3:'Tree', 4:'Car',5: 'Clutter'}
PALETTE = [[255, 255, 255], [0, 0, 255], [0, 255, 255], [0, 255, 0], [255, 204, 0], [255, 0, 0]]

ORIGIN_IMG_SIZE = (256, 256)
INPUT_IMG_SIZE = (256, 256)
TEST_IMG_SIZE = (256, 256)

def get_training_transform():
    train_transform = [
        # albu.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.15),
        # albu.RandomRotate90(p=0.25),
        albu.Normalize()
    ]
    return albu.Compose(train_transform)

def get_val_transform():
    val_transform = [
        albu.RandomRotate90(p=0.5),
        albu.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.5),
        albu.Normalize()
    ]
    return albu.Compose(val_transform)


def val_aug(img, mask):
    img, mask = np.array(img), np.array(mask)
    aug = get_val_transform()(image=img.copy(), mask=mask.copy())
    img, mask = aug['image'], aug['mask']
    return img, mask


class PotsdamDataset(Dataset):
    def __init__(self, data_root='/kaggle/input/deep-data-potsdam-vaihingen/Deep_data_segmentation/Split/Potsdam/', mode='val', img_dir='Image', mask_dir='Label/',
                 img_suffix='.tif', mask_suffix='.png', transform=val_aug, mosaic_ratio=0.0,
                 img_size=ORIGIN_IMG_SIZE):
        self.data_root = data_root
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_suffix = img_suffix
        self.mask_suffix = mask_suffix
        self.transform = transform
        self.mode = mode
        self.mosaic_ratio = mosaic_ratio
        self.img_size = img_size
        self.img_ids = self.get_img_ids(self.data_root, self.img_dir, self.mask_dir)

    def __getitem__(self, index):
        p_ratio = random.random()
        if p_ratio > self.mosaic_ratio or self.mode == 'val' or self.mode == 'test':
            img, mask = self.load_img_and_mask(index)
            if self.transform:
                img, mask = self.transform(img, mask)
        else:
            img, mask = self.load_mosaic_img_and_mask(index)
            if self.transform:
                img, mask = self.transform(img, mask)

        img = torch.from_numpy(img).permute(2, 0, 1).float()
        mask = torch.from_numpy(mask).long()
        img_id = self.img_ids[index]
        results = dict(img_id=img_id, img=img, gt_semantic_seg=mask)
        return results

    def __len__(self):
        return len(self.img_ids)

    def get_img_ids(self, data_root, img_dir, mask_dir):
        img_filename_list = os.listdir(osp.join(data_root, img_dir))
        mask_filename_list = os.listdir(osp.join(data_root, mask_dir))
        assert len(img_filename_list) == len(mask_filename_list)
        img_ids = [str(id.split('.')[0]) for id in mask_filename_list]
        return img_ids

    def load_img_and_mask(self, index):
        img_id = self.img_ids[index]
        img_name = osp.join(self.data_root, self.img_dir, img_id + self.img_suffix)
        mask_name = osp.join(self.data_root, self.mask_dir, img_id + self.mask_suffix)
        img = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('L')
        return img, mask

    def load_mosaic_img_and_mask(self, index):
        indexes = [index] + [random.randint(0, len(self.img_ids) - 1) for _ in range(3)]
        img_a, mask_a = self.load_img_and_mask(indexes[0])
        img_b, mask_b = self.load_img_and_mask(indexes[1])
        img_c, mask_c = self.load_img_and_mask(indexes[2])
        img_d, mask_d = self.load_img_and_mask(indexes[3])

        img_a, mask_a = np.array(img_a), np.array(mask_a)
        img_b, mask_b = np.array(img_b), np.array(mask_b)
        img_c, mask_c = np.array(img_c), np.array(mask_c)
        img_d, mask_d = np.array(img_d), np.array(mask_d)

        w = self.img_size[1]
        h = self.img_size[0]

        start_x = w // 4
        strat_y = h // 4
        # The coordinates of the splice center
        offset_x = random.randint(start_x, (w - start_x))
        offset_y = random.randint(strat_y, (h - strat_y))

        crop_size_a = (offset_x, offset_y)
        crop_size_b = (w - offset_x, offset_y)
        crop_size_c = (offset_x, h - offset_y)
        crop_size_d = (w - offset_x, h - offset_y)

        random_crop_a = albu.RandomCrop(width=crop_size_a[0], height=crop_size_a[1])
        random_crop_b = albu.RandomCrop(width=crop_size_b[0], height=crop_size_b[1])
        random_crop_c = albu.RandomCrop(width=crop_size_c[0], height=crop_size_c[1])
        random_crop_d = albu.RandomCrop(width=crop_size_d[0], height=crop_size_d[1])

        croped_a = random_crop_a(image=img_a.copy(), mask=mask_a.copy())
        croped_b = random_crop_b(image=img_b.copy(), mask=mask_b.copy())
        croped_c = random_crop_c(image=img_c.copy(), mask=mask_c.copy())
        croped_d = random_crop_d(image=img_d.copy(), mask=mask_d.copy())

        img_crop_a, mask_crop_a = croped_a['image'], croped_a['mask']
        img_crop_b, mask_crop_b = croped_b['image'], croped_b['mask']
        img_crop_c, mask_crop_c = croped_c['image'], croped_c['mask']
        img_crop_d, mask_crop_d = croped_d['image'], croped_d['mask']

        top = np.concatenate((img_crop_a, img_crop_b), axis=1)
        bottom = np.concatenate((img_crop_c, img_crop_d), axis=1)
        img = np.concatenate((top, bottom), axis=0)

        top_mask = np.concatenate((mask_crop_a, mask_crop_b), axis=1)
        bottom_mask = np.concatenate((mask_crop_c, mask_crop_d), axis=1)
        mask = np.concatenate((top_mask, bottom_mask), axis=0)
        mask = np.ascontiguousarray(mask)
        img = np.ascontiguousarray(img)

        img = Image.fromarray(img)
        mask = Image.fromarray(mask)

        return img, mask

In [8]:

train_dataset = PotsdamDataset(data_root='/kaggle/input/deep-data-potsdam-vaihingen/Deep_data_segmentation/Split/Potsdam/', mode='train',
                               mosaic_ratio=0.25, transform=val_aug)

train_size = 0.35
valid_size = 0.65
dumb1 = 0.15
dumb2 = 0.85
train_length = round(train_size * len(train_dataset))
valid_length = round(valid_size * len(train_dataset))
train_set, dumb0 = random_split(train_dataset, [train_length, valid_length])
val_set, xxx = random_split(dumb0, [dumb1, dumb2])


test_dataset = PotsdamDataset(data_root='/kaggle/input/deep-data-potsdam-vaihingen/Deep_data_segmentation/Split/Potsdam/Test',
                              transform=val_aug)

train_loader = DataLoader(dataset=train_set,
                          batch_size=batch_size,
                          num_workers=0,
                          pin_memory=False,
                          shuffle=True,
                          drop_last=True
                         )

val_loader = DataLoader(dataset=val_set,
                        batch_size=batch_size,
                        num_workers=2,
                        shuffle=False,
                        pin_memory=True,
                        drop_last=False)


# Metric

In [9]:
class Evaluator(object):
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,) * 2)
        self.eps = 1e-8

    def get_tp_fp_tn_fn(self):
        tp = np.diag(self.confusion_matrix)
        fp = self.confusion_matrix.sum(axis=0) - np.diag(self.confusion_matrix)
        fn = self.confusion_matrix.sum(axis=1) - np.diag(self.confusion_matrix)
        tn = np.diag(self.confusion_matrix).sum() - np.diag(self.confusion_matrix)
        return tp, fp, tn, fn

    def Precision(self):
        tp, fp, tn, fn = self.get_tp_fp_tn_fn()
        precision = tp / (tp + fp)
        return precision

    def Recall(self):
        tp, fp, tn, fn = self.get_tp_fp_tn_fn()
        recall = tp / (tp + fn)
        return recall

    def F1(self):
        tp, fp, tn, fn = self.get_tp_fp_tn_fn()
        Precision = tp / (tp + fp)
        Recall = tp / (tp + fn)
        F1 = (2.0 * Precision * Recall) / (Precision + Recall)
        return F1

    def OA(self):
        OA = np.diag(self.confusion_matrix).sum() / (self.confusion_matrix.sum() + self.eps)
        return OA

    def Intersection_over_Union(self):
        tp, fp, tn, fn = self.get_tp_fp_tn_fn()
        IoU = tp / (tp + fn + fp)
        return IoU

    def Dice(self):
        tp, fp, tn, fn = self.get_tp_fp_tn_fn()
        Dice = 2 * tp / ((tp + fp) + (tp + fn))
        return Dice

    def Pixel_Accuracy_Class(self):
        #         TP                                  TP+FP
        Acc = np.diag(self.confusion_matrix) / (self.confusion_matrix.sum(axis=0) + self.eps)
        return Acc

    def Frequency_Weighted_Intersection_over_Union(self):
        freq = np.sum(self.confusion_matrix, axis=1) / (np.sum(self.confusion_matrix) + self.eps)
        iou = self.Intersection_over_Union()
        FWIoU = (freq[freq > 0] * iou[freq > 0]).sum()
        return FWIoU

    def _generate_matrix(self, gt_image, pre_image):
        mask = (gt_image >= 0) & (gt_image < self.num_class)
        label = self.num_class * gt_image[mask].astype('int') + pre_image[mask]
        count = np.bincount(label, minlength=self.num_class ** 2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape, 'pre_image shape {}, gt_image shape {}'.format(pre_image.shape,
                                                                                                 gt_image.shape)
        self.confusion_matrix += self._generate_matrix(gt_image, pre_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

In [10]:
!pip install einops

  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.0 MB/s eta 0:00:00


# Model

In [11]:
# import torch

# # Instantiate your UNet model
# model = A2FPN()


# # Print the shape of the outputmodel = UNet(n_class=6)

# # Create a dummy batch with random images
# dummy_batch_size = 4  # Choose your batch size
# dummy_input_batch = torch.randn(dummy_batch_size, 3, 256, 256)  # Batch size, 3 channels, 256x256

# # Forward pass
# output_batch = model(dummy_input_batch)

# # Print the shape of the output batch

# print("Output batch shape:", output_batch.shape)

# Loss function

In [12]:
def label_smoothed_nll_loss(
    lprobs: torch.Tensor, target: torch.Tensor, epsilon: float, ignore_index=None, reduction="mean", dim=-1
) -> torch.Tensor:
    """

    Source: https://github.com/pytorch/fairseq/blob/master/fairseq/criterions/label_smoothed_cross_entropy.py

    :param lprobs: Log-probabilities of predictions (e.g after log_softmax)
    :param target:
    :param epsilon:
    :param ignore_index:
    :param reduction:
    :return:
    """
    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(dim)

    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        target = target.masked_fill(pad_mask, 0)
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        # nll_loss.masked_fill_(pad_mask, 0.0)
        # smooth_loss.masked_fill_(pad_mask, 0.0)
        nll_loss = nll_loss.masked_fill(pad_mask, 0.0)
        smooth_loss = smooth_loss.masked_fill(pad_mask, 0.0)
    else:
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        nll_loss = nll_loss.squeeze(dim)
        smooth_loss = smooth_loss.squeeze(dim)

    if reduction == "sum":
        nll_loss = nll_loss.sum()
        smooth_loss = smooth_loss.sum()
    if reduction == "mean":
        nll_loss = nll_loss.mean()
        smooth_loss = smooth_loss.mean()

    eps_i = epsilon / lprobs.size(dim)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss

In [13]:
__all__ = ["SoftCrossEntropyLoss"]
from typing import Optional
from torch.nn.modules.loss import _Loss
from typing import List
class SoftCrossEntropyLoss(nn.Module):
    """
    Drop-in replacement for nn.CrossEntropyLoss with few additions:
    - Support of label smoothing
    """

    __constants__ = ["reduction", "ignore_index", "smooth_factor"]

    def __init__(self, reduction: str = "mean", smooth_factor: float = 0.0, ignore_index: Optional[int] = -100, dim=1):
        super().__init__()
        self.smooth_factor = smooth_factor
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.dim = dim

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        log_prob = F.log_softmax(input, dim=self.dim)
        pad_mask = target.eq(self.ignore_index)
        target = target.masked_fill(pad_mask, 0)
        log_prob = log_prob.masked_fill(pad_mask.unsqueeze(1), 0)
        return label_smoothed_nll_loss(
            log_prob,
            target,
            epsilon=self.smooth_factor,
            ignore_index=self.ignore_index,
            reduction=self.reduction,
            dim=self.dim,
        )
__all__ = ["JointLoss", "WeightedLoss"]


class WeightedLoss(_Loss):
    """Wrapper class around loss function that applies weighted with fixed factor.
    This class helps to balance multiple losses if they have different scales
    """

    def __init__(self, loss, weight=1.0):
        super().__init__()
        self.loss = loss
        self.weight = weight

    def forward(self, *input):
        return self.loss(*input) * self.weight
class JointLoss(_Loss):
    """
    Wrap two loss functions into one. This class computes a weighted sum of two losses.
    """

    def __init__(self, first: nn.Module, second: nn.Module, first_weight=1.0, second_weight=1.0):
        super().__init__()
        self.first = WeightedLoss(first, first_weight)
        self.second = WeightedLoss(second, second_weight)

    def forward(self, *input):
        return self.first(*input) + self.second(*input)
__all__ = ["DiceLoss"]

BINARY_MODE = "binary"
MULTICLASS_MODE = "multiclass"
MULTILABEL_MODE = "multilabel"
def soft_dice_score(
    output: torch.Tensor, target: torch.Tensor, smooth: float = 0.0, eps: float = 1e-7, dims=None
) -> torch.Tensor:
    """

    :param output:
    :param target:
    :param smooth:
    :param eps:
    :return:

    Shape:
        - Input: :math:`(N, NC, *)` where :math:`*` means any number
            of additional dimensions
        - Target: :math:`(N, NC, *)`, same shape as the input
        - Output: scalar.

    """
    assert output.size() == target.size()
    if dims is not None:
        intersection = torch.sum(output * target, dim=dims)
        cardinality = torch.sum(output + target, dim=dims)
    else:
        intersection = torch.sum(output * target)
        cardinality = torch.sum(output + target)
    dice_score = (2.0 * intersection + smooth) / (cardinality + smooth).clamp_min(eps)
    return dice_score
class DiceLoss(_Loss):
    """
    Implementation of Dice loss for image segmentation task.
    It supports binary, multiclass and multilabel cases
    """

    def __init__(
        self,
        mode: str = 'multiclass',
        classes: List[int] = None,
        log_loss=False,
        from_logits=True,
        smooth: float = 0.0,
        ignore_index=None,
        eps=1e-7,
    ):
        """

        :param mode: Metric mode {'binary', 'multiclass', 'multilabel'}
        :param classes: Optional list of classes that contribute in loss computation;
        By default, all channels are included.
        :param log_loss: If True, loss computed as `-log(jaccard)`; otherwise `1 - jaccard`
        :param from_logits: If True assumes input is raw logits
        :param smooth:
        :param ignore_index: Label that indicates ignored pixels (does not contribute to loss)
        :param eps: Small epsilon for numerical stability
        """
        assert mode in {BINARY_MODE, MULTILABEL_MODE, MULTICLASS_MODE}
        super(DiceLoss, self).__init__()
        self.mode = mode
        if classes is not None:
            assert mode != BINARY_MODE, "Masking classes is not supported with mode=binary"
            classes = to_tensor(classes, dtype=torch.long)

        self.classes = classes
        self.from_logits = from_logits
        self.smooth = smooth
        self.eps = eps
        self.ignore_index = ignore_index
        self.log_loss = log_loss

    def forward(self, y_pred: Tensor, y_true: Tensor) -> Tensor:
        """

        :param y_pred: NxCxHxW
        :param y_true: NxHxW
        :return: scalar
        """
        assert y_true.size(0) == y_pred.size(0)

        if self.from_logits:
            # Apply activations to get [0..1] class probabilities
            # Using Log-Exp as this gives more numerically stable result and does not cause vanishing gradient on
            # extreme values 0 and 1
            if self.mode == MULTICLASS_MODE:
                y_pred = y_pred.log_softmax(dim=1).exp()
            else:
                y_pred = F.logsigmoid(y_pred).exp()

        bs = y_true.size(0)
        num_classes = y_pred.size(1)
        dims = (0, 2)

        if self.mode == BINARY_MODE:
            y_true = y_true.view(bs, 1, -1)
            y_pred = y_pred.view(bs, 1, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask
                y_true = y_true * mask

        if self.mode == MULTICLASS_MODE:
            y_true = y_true.view(bs, -1)
            y_pred = y_pred.view(bs, num_classes, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask.unsqueeze(1)

                y_true = F.one_hot((y_true * mask).to(torch.long), num_classes)  # N,H*W -> N,H*W, C
                y_true = y_true.permute(0, 2, 1) * mask.unsqueeze(1)  # H, C, H*W
            else:
                y_true = F.one_hot(y_true, num_classes)  # N,H*W -> N,H*W, C
                y_true = y_true.permute(0, 2, 1)  # H, C, H*W

        if self.mode == MULTILABEL_MODE:
            y_true = y_true.view(bs, num_classes, -1)
            y_pred = y_pred.view(bs, num_classes, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask
                y_true = y_true * mask

        scores = soft_dice_score(y_pred, y_true.type_as(y_pred), smooth=self.smooth, eps=self.eps, dims=dims)

        if self.log_loss:
            loss = -torch.log(scores.clamp_min(self.eps))
        else:
            loss = 1.0 - scores

        # Dice loss is undefined for non-empty classes
        # So we zero contribution of channel that does not have true pixels
        # NOTE: A better workaround would be to use loss term `mean(y_pred)`
        # for this case, however it will be a modified jaccard loss

        mask = y_true.sum(dims) > 0
        loss *= mask.to(loss.dtype)

        if self.classes is not None:
            loss = loss[self.classes]

        return loss.mean()


In [14]:

class UnetLoss(nn.Module):
    def __init__(self, ignore_index=255):
        super().__init__()
        self.main_loss = JointLoss(SoftCrossEntropyLoss(smooth_factor=0.05, ignore_index=ignore_index),
                                   DiceLoss(smooth=0.05, ignore_index=ignore_index), 1.0, 1.0)
        self.aux_loss = SoftCrossEntropyLoss(smooth_factor=0.05, ignore_index=ignore_index)

    def forward(self, logits, labels):
        if self.training and len(logits) == 2:
            logit_main, logit_aux = logits
            loss = self.main_loss(logit_main, labels) + 0.4 * self.aux_loss(logit_aux, labels)
        else:
            loss = self.main_loss(logits, labels)

        return loss

# Training

**Initialize weights**

In [15]:
def weights_init(model):
    if isinstance(model, nn.Linear):
        # Xavier Distribution
        torch.nn.init.xavier_uniform_(model.weight)

In [16]:
def save_model(model, optimizer, path):
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, path)

def load_model(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer

**Train model**

In [17]:
def train(train_dataloader, valid_dataloader, learing_rate_scheduler, epoch, display_step):
    print(f"Start epoch #{epoch+1}, learning rate for this epoch: {learing_rate_scheduler.get_last_lr()}")
    start_time = time.time()
    train_loss_epoch = 0
    test_loss_epoch = 0
    last_loss = 999999999
    model.train()
    metrics_train = Evaluator(num_class=6)
    metrics_val = Evaluator(num_class=6)

    for i,input in enumerate(tqdm(train_dataloader)):
        # Load data into GPU
        data=input['img']
        masks_true = input['gt_semantic_seg']
        data,mask = data.to(device),masks_true.to(device)
        optimizer.zero_grad()
        prediction = model(data)
        # Backpropagation, compute gradients
        loss = loss_function(prediction, mask.long())
        pre_mask = nn.Softmax(dim=1)(prediction)
        pre_mask = pre_mask.argmax(dim=1)
        for i in range(mask.shape[0]):
            metrics_train.add_batch(mask[i].cpu().numpy(), pre_mask[i].cpu().numpy())
        loss.backward()

        # Apply gradients
        optimizer.step()

        # Save loss
        train_loss_epoch += loss.item()
    print(f"Done epoch #{epoch+1}, time for this epoch: {time.time()-start_time}s")
    train_loss_epoch /= (i + 1)
    mIoU = np.nanmean(metrics_train.Intersection_over_Union()[:-1])
    F1 = np.nanmean(metrics_train.F1()[:-1])
    OA = np.nanmean(metrics_train.OA())
    iou_per_class = metrics_train.Intersection_over_Union()
    train_eval_value =  (iou_per_class,mIoU,F1,OA)
    metrics_train.reset()
    # Evaluate the validation set
    model.eval()
    with torch.no_grad():
        for k,input in enumerate(tqdm(valid_dataloader)):
            data=input['img'].to(device)
            mask = input['gt_semantic_seg'].to(device)
            prediction = model(data)
            test_loss = loss_function(prediction, mask.long())
            pre_mask = nn.Softmax(dim=1)(prediction)
            pre_mask = pre_mask.argmax(dim=1)
            test_loss_epoch += test_loss.item()
            if k<=3:
            # Convert predictions to 2D array
                predictions_2d = pre_mask[0].cpu().numpy()  # Assuming you want the first prediction

            # Convert ground truth masks to 2D array
                masks_true_2d = mask[0].cpu().numpy()  # Assuming you want the first ground truth mask

            # Convert to np.int8 if needed
                predictions_2d = predictions_2d.astype(np.int8)
                masks_true_2d = masks_true_2d.astype(np.int8)
                wandb.log(
      {f"val_image{k}" : wandb.Image(data[0], masks={
        "predictions" : {
            "mask_data" : predictions_2d,
            "class_labels" : class_label
        },
        "ground_truth" : {
            "mask_data" : masks_true_2d,
            "class_labels" : class_label
        }
    })})
            for i in range(mask.shape[0]):
                metrics_val.add_batch(mask[i].cpu().numpy(), pre_mask[i].cpu().numpy())
    test_loss_epoch /= (i + 1)
    mIoU = np.nanmean(metrics_val.Intersection_over_Union()[:-1])
    F1 = np.nanmean(metrics_val.F1()[:-1])
    OA = np.nanmean(metrics_val.OA())
    iou_per_class_val = metrics_val.Intersection_over_Union()
    eval_value =  (iou_per_class_val,mIoU,F1,OA)
    print(eval_value)
    metrics_val.reset()
    return train_loss_epoch, train_eval_value, test_loss_epoch, eval_value


**Test model**

In [18]:
device

device(type='cuda')

In [19]:
!pip install segmentation-models-pytorch

  Obtaining dependency information for segmentation-models-pytorch from https://files.pythonhosted.org/packages/cb/70/4aac1b240b399b108ce58029ae54bc14497e1bbc275dfab8fd3c84c1e35d/segmentation_models_pytorch-0.3.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Preparing metadata (setup.py) ... - done
  Obtaining dependency information for timm==0.9.2 from https://files.pythonhosted.org/packages/29/90/94f5deb8d76e24a89813aef95e8809ca8fd7414490428480eda19b133d4a/timm-0.9.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 6.6 MB/s eta 0:00:00
  Obtaining dependency information for munch from https://files.pythonhosted.org/packages/56/b3/7c69b37f03260a061883bec0e7b05be7117c1b1c85f5212c72c8c2bc3c8c/munch-4.0.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name="resnet34",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=6     
)

model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 219MB/s]


UnetPlusPlus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [21]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Use GPU-accelerated PyTorch functions here
    print("CUDA is available.")
else:
    print("CUDA is not available.")

CUDA is available.


In [22]:

loss_function = UnetLoss(ignore_index=6)
# Define the optimizer (Adam optimizer)
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
# optimizer.load_state_dict(checkpoint['optimizer'])

# Learning rate scheduler
learing_rate_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.6)

In [23]:
save_model(model, optimizer, checkpoint_path)
load_checkpoint_flag=False

In [24]:
# print("Model keys:")
# print(model.state_dict().keys())

# print("\nCheckpoint keys:")
# print(checkpoint['model'].keys())


In [25]:
# Load the model checkpoint if needed
# Load the model checkpoint if needed
if load_checkpoint_flag:
    model, optimize= load_model(model, optimizer, pretrained_path)
    


In [26]:
torch.cuda.empty_cache()

In [27]:

wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment", 
    key = "b98d2b806f364f5af900550ec98e26e2f418e8a7",
)
wandb.init(
    project = "unetplusplus_tuning"
)



wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: trithuy2003 (deepbk). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231216_114033-ubeetgwe
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run helpful-eon-9
wandb: ⭐️ View project at https://wandb.ai/deepbk/unetplusplus_tuning
wandb: 🚀 View run at https://wandb.ai/deepbk/unetplusplus_tuning/runs/ubeetgwe


In [28]:

# Training loop
train_loss_array = []
test_loss_array = []
last_loss = 9999999999999
for epoch in range(40):
    try:
        train_loss_epoch = 0
        test_loss_epoch = 0
        train_loss_epoch, train_eval_value, test_loss_epoch, eval_value = train(train_loader, 
                                                  val_loader, 
                                                  learing_rate_scheduler, epoch, display_step)

        if test_loss_epoch < last_loss:
            save_model(model, optimizer, checkpoint_path)
            last_loss = test_loss_epoch

        iou_value = {}
        iou_per_class,mIoU,F1,OA=train_eval_value
        eval_value_train = {'mIoU': mIoU,
                          'F1': F1,
                          'OA': OA}                                          
        print('train:', eval_value_train)
        train_accuracy.append(OA)
        wandb.log({'mIoU_train': mIoU,
                          'F1_train': F1,
                          'OA_train': OA}) 

        for class_name, iou in zip(CLASSES,iou_per_class):
            wandb.log({f"{class_name}_train_IOU": iou})
            iou_value[class_name] = iou
        print(iou_value)

        iou_value = {}
        iou_per_class,mIoU,F1,OA = eval_value
        eval_value_val = {'mIoU': mIoU,
                          'F1': F1,
                          'OA': OA}                                          
        print('val:', eval_value_val)
        valid_accuracy.append(OA)
        wandb.log({'mIoU_val': mIoU,
                          'F1_val': F1,
                          'OA_val': OA})
        for class_name, iou in zip(CLASSES,iou_per_class):
            wandb.log({f"{class_name}_val_IoU": iou})
            iou_value[class_name] = iou
        print(iou_value)

        learing_rate_scheduler.step()
        train_loss_array.append(train_loss_epoch)
        test_loss_array.append(test_loss_epoch)
        wandb.log({"Train loss": train_loss_epoch, "Valid loss": test_loss_epoch})
        print("Epoch {}: loss: {:.4f}, train accuracy: {:.4f}, valid accuracy:{:.4f}".format(epoch + 1, 
                                            train_loss_array[-1], train_accuracy[-1], valid_accuracy[-1]))
    except:
        torch.cuda.empty_cache()


Start epoch #1, learning rate for this epoch: [0.001]


100%|██████████| 312/312 [04:05<00:00,  1.27it/s]


Done epoch #1, time for this epoch: 245.98548126220703s


100%|██████████| 88/88 [00:20<00:00,  4.20it/s]


(array([0.49148501, 0.39317315, 0.22669158, 0.29615695, 0.5553317 ,
       0.04467074]), 0.39256767777341944, 0.5528317637345344, 0.5238985169815314)
train: {'mIoU': 0.4532835015379728, 'F1': 0.6207107097769999, 'OA': 0.6372596331131763}
{'ImSurf': 0.4925088031720075, 'Building': 0.5095192321219401, 'LowVeg': 0.47614477805557953, 'Tree': 0.4649576011363042, 'Car': 0.32328709320403265, 'Clutter': 0.03316370096161863}
val: {'mIoU': 0.39256767777341944, 'F1': 0.5528317637345344, 'OA': 0.5238985169815314}
{'ImSurf': 0.49148501014474455, 'Building': 0.3931731522834272, 'LowVeg': 0.22669157805251644, 'Tree': 0.2961569473463578, 'Car': 0.555331701040051, 'Clutter': 0.044670744935739894}
Epoch 1: loss: 32.4185, train accuracy: 0.6373, valid accuracy:0.5239
Start epoch #2, learning rate for this epoch: [0.001]


...

100%|██████████| 88/88 [00:12<00:00,  7.26it/s]


(array([0.74476496, 0.85755754, 0.70726967, 0.70555443, 0.75082271,
       0.45759052]), 0.7531938618161227, 0.8581221422084369, 0.8488146582697246)
train: {'mIoU': 0.7897679358310787, 'F1': 0.8813953835629352, 'OA': 0.8775718059295263}
{'ImSurf': 0.8015007104570863, 'Building': 0.8957850335396774, 'LowVeg': 0.7359805224695358, 'Tree': 0.7417244179187341, 'Car': 0.7738489947703604, 'Clutter': 0.5272988605215414}
val: {'mIoU': 0.7531938618161227, 'F1': 0.8581221422084369, 'OA': 0.8488146582697246}
{'ImSurf': 0.7447649640333002, 'Building': 0.857557540076613, 'LowVeg': 0.7072696697934135, 'Tree': 0.705554429920706, 'Car': 0.7508227052565809, 'Clutter': 0.4575905213037916}
Epoch 39: loss: 15.5396, train accuracy: 0.8776, valid accuracy:0.8488
Start epoch #40, learning rate for this epoch: [1.0077695999999996e-05]


100%|██████████| 312/312 [02:24<00:00,  2.16it/s]


Done epoch #40, time for this epoch: 144.61492228507996s


  5%|▍         | 4/88 [00:00<00:16,  4.98it/s]Exception in thread Thread-48 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 51, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.10/site-packages/torch/multiprocessing/reductions.py", line 307, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.10/multiprocessing/resource_sharer.py", line 58, in detach
    return red

In [29]:
torch.cuda.empty_cache()

In [30]:
def img_writer(inp):
    (mask,  mask_id, rgb) = inp
    if rgb:
        mask_name_tif = mask_id + '.png'
        mask_tif = label2rgb(mask)
        cv2.imwrite(mask_name_tif, mask_tif)
    else:
        mask_png = mask.astype(np.uint8)
        mask_name_png = mask_id + '.png'
        cv2.imwrite(mask_name_png, mask_png)


def get_args():
    parser = argparse.ArgumentParser()
    arg = parser.add_argument
    arg("-c", "--config_path", type=Path, required=True, help="Path to  config")
    arg("-o", "--output_path", type=Path, help="Path where to save resulting masks.", required=True)
    arg("-t", "--tta", help="Test time augmentation.", default=None, choices=[None, "d4", "lr"])
    arg("--rgb", help="whether output rgb images", action='store_true')
    return parser.parse_args()

In [31]:
test_dataset = PotsdamDataset(data_root='/kaggle/input/deep-data-potsdam-vaihingen/Deep_data_segmentation/Split/Vaihingen/Test',
                              transform=val_aug)

In [32]:
model.cuda()
model.eval()
evaluator = Evaluator(num_class=6)
evaluator.reset()
with torch.no_grad():
    test_loader = DataLoader(
        test_dataset,
        batch_size=2,
        num_workers=4,
        pin_memory=True,
        drop_last=False,
    )
    results = []
    for input in tqdm(test_loader):
        # raw_prediction NxCxHxW
        raw_predictions = model(input['img'].cuda())

        image_ids = input["img_id"]
        masks_true = input['gt_semantic_seg']

        raw_predictions = nn.Softmax(dim=1)(raw_predictions)
        predictions = raw_predictions.argmax(dim=1)

        for i in range(raw_predictions.shape[0]):
            mask = predictions[i].cpu().numpy()
            evaluator.add_batch(pre_image=mask, gt_image=masks_true[i].cpu().numpy())
            mask_name = image_ids[i]
iou_per_class = evaluator.Intersection_over_Union()
f1_per_class = evaluator.F1()
OA = evaluator.OA()
for class_name, class_iou, class_f1 in zip(CLASSES, iou_per_class, f1_per_class):
    print('F1_{}:{}, IOU_{}:{}'.format(class_name, class_f1, class_name, class_iou))
print('F1:{}, mIOU:{}, OA:{}'.format(np.nanmean(f1_per_class[:-1]), np.nanmean(iou_per_class[:-1]), OA))
t0 = time.time()
mpp.Pool(processes=2).map(img_writer, results)
t1 = time.time()
img_write_time = t1 - t0
print('images writing spends: {} s'.format(img_write_time))

100%|██████████| 235/235 [00:05<00:00, 40.25it/s]

F1_ImSurf:0.5618505399074194, IOU_ImSurf:0.39067604271898865
F1_Building:0.7103269497812126, IOU_Building:0.5507806413886907
F1_LowVeg:0.06093045547741564, IOU_LowVeg:0.031422522028428455
F1_Tree:0.04499079610776339, IOU_Tree:0.023013086597337244
F1_Car:0.2496819635751765, IOU_Car:0.14264948333913852
F1_Clutter:0.08056004587906423, IOU_Clutter:0.0419705996564811
F1:0.3255561409697975, mIOU:0.2277083552145167, OA:0.4430487450132977


images writing spends: 0.05967283248901367 s
